# On the Identification of Pulsar Stars

A DSCI 100 Project

Lucas Kuhn, Sophia Zhang, Michael Cheung

## Introduction

Out in the universe, astronomers have found many neutron stars. A subset of these stars fall under a rare classification known as a "pulsar", meaning that they produce a periodic radio signal that is detectable from Earth. These stars are of great interest, and so being able to identify them is a useful skill that furthers exploration efforts in understanding neutron stars.

In this project, we set out to develop a model for classifying them distinctly from those that are "non-pulsar". Our question to answer: "Are there characteristics of a pulsar candidate's curve profiles that serve as good predictors for pulsar and non-pulsar classification?".

To investigate this, we use the HTRU2 dataset provided by the UCI Machine Learning Repository. This dataset contains summary statistics for a large number of observed pulsar candidates as they exhibit on both the Integrated Pulse (folded) Profile as well as on the Dispersion Measure Signal-to-Noise Ratio (DM-SNR) curve, which is then associated alongside their known classifications as either "pulsar" or "non-pulsar".

## Methods

To carry out the investigation, we will train multiple K-Nearest Neighbours classifier models using various subsets of the columns in our dataset as predictor variables. Given how the candidates that are truly pulsar generally broadcast signal pulses that are "remarkably constant over long periods of time" (Kastergiou et. al, 2011), we can reasonably surmise that the summary statistics from candidates' curve profiles could serve as predictors for our classification model.

As a way of gaining insights as to which dataset columns to use in our set of predictor columns, we will first create scatter plots of pairs of the columns in a scatter plot matrix, colouring the points by their classification and inspecting for visible divisions. For example, columns that we may wish to exclude from the predictor variables are those that result in "heterogeneous" mixtures of the classified points with less defined independent groupings of points. This is a valid way to evaluate for predictors as plots provide an interface for human inspection, and are used to train other models such as artificial neural networks (Eatough et. al, 2010).

Using a GridSearchCV with 5-fold Cross-Validation, we can optimize the number of neighbours $k$. This allows us to efficiently test the validation accuracy at many values of $k$ and combat both overfitting and underfitting.

With our best model, we can assess its accuracy on the testing set using a confusion matrix, allowing us to more critically assess the accuracy based on whether false positives are more costly than false negatives.

## Results

We perform preliminary setup and retrieve the packages we expect to use.

In [1]:
# Retrieve relevant packages for dataframes and visualization tools
import pandas as pd
import altair as alt

# Retrieve relevant packages for classification and modelling
from sklearn.compose import make_column_transformer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import (
    GridSearchCV,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

As the dataset we plan to import will contain more than 5000 observations, we will need to allow for greater data plotting.

In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

The HTRU2 dataset can be downloaded from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/machine-learning-databases/00372/). We unzip the file to the `data/` directory and read the corresponding .CSV file into Jupyter Notebook.

In [3]:
pulsar_data = pd.read_csv(
    "../data/HTRU2/HTRU_2.csv",
    # .CSV file contains no column headers, we provide these here
    names=[
        "Profile_mean",
        "Profile_stdev",
        "Profile_skewness",
        "Profile_kurtosis",
        "DM_mean",
        "DM_stdev",
        "DM_skewness",
        "DM_kurtosis",
        "class"
    ]
)
pulsar_data.head()

,Profile_mean,Profile_stdev,Profile_skewness,Profile_kurtosis,DM_mean,DM_stdev,DM_skewness,DM_kurtosis,class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0


At a high-level, we query for properties of the dataset's shape and columns.

In [4]:
pulsar_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17898 entries, 0 to 17897
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Profile_mean      17898 non-null  float64
 1   Profile_stdev     17898 non-null  float64
 2   Profile_skewness  17898 non-null  float64
 3   Profile_kurtosis  17898 non-null  float64
 4   DM_mean           17898 non-null  float64
 5   DM_stdev          17898 non-null  float64
 6   DM_skewness       17898 non-null  float64
 7   DM_kurtosis       17898 non-null  float64
 8   class             17898 non-null  int64  
dtypes: float64(8), int64(1)
memory usage: 1.2 MB


From inspection of the first 5 rows, we can see that the dataset is already tidy, as it contains variables as columns, single candidates (observations) as rows, and each cell of the table containing a single numerical value. Furthermore, we note that the dataset contains 17,898 entries, and that there are no null values throughout, since all columns have the same number of non-null values.

The "class" column currently contains a simple integer value: 0 for non-pulsar, and 1 for pulsar. For ease of identification, we change these to string equivalents for easier interpretability.

In [5]:
# Include a call to the `convert_dtypes()` method to change to a more suitable data type string
pulsar_data["class"] = pulsar_data["class"].replace({0: "non-pulsar", 1: "pulsar"}).convert_dtypes()
display(pulsar_data.info())
pulsar_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17898 entries, 0 to 17897
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Profile_mean      17898 non-null  float64
 1   Profile_stdev     17898 non-null  float64
 2   Profile_skewness  17898 non-null  float64
 3   Profile_kurtosis  17898 non-null  float64
 4   DM_mean           17898 non-null  float64
 5   DM_stdev          17898 non-null  float64
 6   DM_skewness       17898 non-null  float64
 7   DM_kurtosis       17898 non-null  float64
 8   class             17898 non-null  string 
dtypes: float64(8), string(1)
memory usage: 1.2 MB


None

,Profile_mean,Profile_stdev,Profile_skewness,Profile_kurtosis,DM_mean,DM_stdev,DM_skewness,DM_kurtosis,class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,non-pulsar
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,non-pulsar
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,non-pulsar
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,non-pulsar
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,non-pulsar
...,...,...,...,...,...,...,...,...,...
17893,136.429688,59.847421,-0.187846,-0.738123,1.296823,12.166062,15.450260,285.931022,non-pulsar
17894,122.554688,49.485605,0.127978,0.323061,16.409699,44.626893,2.945244,8.297092,non-pulsar
17895,119.335938,59.935939,0.159363,-0.743025,21.430602,58.872000,2.499517,4.595173,non-pulsar
17896,114.507812,53.902400,0.201161,-0.024789,1.946488,13.381731,10.007967,134.238910,non-pulsar


Our training data should include non-pulsar and pulsar observations to best inform the model. Our testing data should follow the same idea to provide a fair test of accuracy of the model.

In [6]:
# Keep one other column outside the target classification column
pulsar_data_count = pulsar_data[["class", "Profile_mean"]].groupby("class").count().reset_index()

# Rename the column to more accurately reflect its purpose
pulsar_data_count = pulsar_data_count.rename(columns={"Profile_mean": "count"})
pulsar_data_count = pulsar_data_count.assign(pct=pulsar_data_count["count"] / len(pulsar_data))
pulsar_data_count

,class,count,pct
0,non-pulsar,16259,0.908426
1,pulsar,1639,0.091574


We will base the proportions on what is found in the full version of the data. We will also arbitrarily use 75% of the data for training, and the remainder for testing.

In [7]:
pulsar_data_train, pulsar_data_test = train_test_split(
    pulsar_data, train_size=0.75, stratify=pulsar_data["class"]
)

pulsar_data_train_count = pulsar_data_train[["class", "Profile_mean"]].groupby("class").count().reset_index()
pulsar_data_train_count = pulsar_data_train_count.rename(columns={"Profile_mean": "count"})
pulsar_data_train_count = pulsar_data_train_count.assign(pct=pulsar_data_train_count["count"] / len(pulsar_data_train))

# We only evaluate the distribution of the testing set; we seek no further detail into the nature of the testing set past this
pulsar_data_test_count = pulsar_data_test[["class", "Profile_mean"]].groupby("class").count().reset_index()
pulsar_data_test_count = pulsar_data_test_count.rename(columns={"Profile_mean": "count"})
pulsar_data_test_count = pulsar_data_test_count.assign(pct=pulsar_data_test_count["count"] / len(pulsar_data_test))

In [8]:
pulsar_data_train_count

,class,count,pct
0,non-pulsar,12194,0.908441
1,pulsar,1229,0.091559


In [9]:
pulsar_data_test_count

,class,count,pct
0,non-pulsar,4065,0.90838
1,pulsar,410,0.09162


The full dataset had a distribution of around 91% of the observations as "non-pulsar", and about 9% as "pulsar". The training and testing set are thus stratified to maintain this distribution.

Using our training data, we create our scatter plot matrix of all of our columns, so we can assess which variables may be good predictors.

In [10]:
pulsar_data_scatter_plot_matrix = (
    alt.Chart(pulsar_data_train)
    .mark_point(opacity=0.3)
    .encode(
        x=alt.X(alt.repeat("column"), type="quantitative"),
        y=alt.Y(alt.repeat("row"), type="quantitative"),
        color="class",
    )
    .properties(width=100, height=100)
    .repeat(
        row=[
            "Profile_mean", "Profile_stdev", "Profile_skewness", "Profile_kurtosis",
            "DM_mean", "DM_stdev", "DM_skewness", "DM_kurtosis"
        ],
        column=[
            "DM_kurtosis", "DM_skewness", "DM_stdev", "DM_mean",
            "Profile_kurtosis", "Profile_skewness", "Profile_stdev", "Profile_mean"
        ]
    )
)

pulsar_data_scatter_plot_matrix

alt.RepeatChart(...)

From this scatter plot matrix, we can see that our best predictors are likely those that consistently produce visible distinctions in the "non-pulsar" and "pulsar" classifications, with little to no "mixing" of categories.

With this as our definition, we suggest that our set of variables to use as predictors should contain:
- `Profile_mean`
- `Profile_skewness`
- `Profile_kurtosis`

We exclude the `Profile_stdev` as well as the statistics from the DM-SNR curve as they produce multiple plots with a significant mixing of points from both categories (e.g. plots within the lower-left section of the matrix).

With these 3 variables, we begin creating the pipelines for building our classifier models. As we are evaluating euclidean distances between points within a K-Nearest Neighbours model, and the predictors we intend to use have different value ranges (i.e. `Profile_mean` may have narrower/wider range than `Profile_skewness`), we will have to scale our values through preprocessing.

To build models using the different subset combinations of the predictor set, we will create multiple preprocessors along with the same number of pipelines.

In [11]:
# One predictor
mean_preprocessor = make_column_transformer(
    (StandardScaler(), ["Profile_mean"]),
)

skewness_preprocessor = make_column_transformer(
    (StandardScaler(), ["Profile_skewness"]),
)

kurtosis_preprocessor = make_column_transformer(
    (StandardScaler(), ["Profile_kurtosis"]),   
)

# Two predictors
mean_skewness_preprocessor = make_column_transformer(
    (StandardScaler(), ["Profile_mean", "Profile_skewness"]),
)

mean_kurtosis_preprocessor = make_column_transformer(
    (StandardScaler(), ["Profile_mean", "Profile_kurtosis"]),
)

skewness_kurtosis_preprocessor = make_column_transformer(
    (StandardScaler(), ["Profile_skewness", "Profile_kurtosis"]),
)

# All predictors
all_preprocessor = make_column_transformer(
    (StandardScaler(), ["Profile_mean", "Profile_skewness", "Profile_kurtosis"]),
)

Now that we have a preprocessor for each subset of the intended predictors, we can make their corresponding pipeline objects.

In [12]:
# One predictor
mean_pipe = make_pipeline(mean_preprocessor, KNeighborsClassifier())

skewness_pipe = make_pipeline(skewness_preprocessor, KNeighborsClassifier())

kurtosis_pipe = make_pipeline(kurtosis_preprocessor, KNeighborsClassifier())

# Two predictors
mean_skewness_pipe = make_pipeline(mean_skewness_preprocessor, KNeighborsClassifier())

mean_kurtosis_pipe = make_pipeline(mean_kurtosis_preprocessor, KNeighborsClassifier())

skewness_kurtosis_pipe = make_pipeline(skewness_kurtosis_preprocessor, KNeighborsClassifier())

# All predictors
all_pipe = make_pipeline(all_preprocessor, KNeighborsClassifier())

To determine an optimal hyperparameter $k$ to use as the number of neighbours in the KNN classifier model, we will run a `GridSearchCV` which will give us an idea of the best value. We will first set up the data that will be used to fit each run of `GridSearchCV`; the `X` and `y` data resulting from the preprocessors.

The preprocessors will already drop any columns not marked as part of the `StandardScaler()` by default; as such, we only need to ensure that `X` and `y` split up the training labels.

In [13]:
X_train = pulsar_data_train.drop(columns=["class"])
y_train = pulsar_data_train["class"]

With that, we can apply a `GridSearchCV` with each of the pipelines, fitting them to the `X_train` and `y_train` components. We will set up our search to investigate accuracies using a number of neighbours from 1 to 15. We will use a default 5-fold cross-validation.

In [14]:
# Create parameter grid for number of neighbours to evaluate models on
param_grid = {
    "kneighborsclassifier__n_neighbors": range(1, 15, 1),
}

In [15]:
# For each pipe, create a GridSearchCV and fit the data

mean_grid = GridSearchCV(
    estimator=mean_pipe,
    param_grid=param_grid,
    cv=5
)
mean_model_grid = mean_grid.fit(X_train, y_train)


skewness_grid = GridSearchCV(
    estimator=skewness_pipe,
    param_grid=param_grid,
    cv=5,
)
skewness_model_grid = skewness_grid.fit(X_train, y_train)


kurtosis_grid = GridSearchCV(
    estimator=kurtosis_pipe,
    param_grid=param_grid,
    cv=5,
)
kurtosis_model_grid = kurtosis_grid.fit(X_train, y_train)


mean_skewness_grid = GridSearchCV(
    estimator=mean_skewness_pipe,
    param_grid=param_grid,
    cv=5
)
mean_skewness_model_grid = mean_skewness_grid.fit(X_train, y_train)


mean_kurtosis_grid = GridSearchCV(
    estimator=mean_kurtosis_pipe,
    param_grid=param_grid,
    cv=5,
)
mean_kurtosis_model_grid = mean_kurtosis_grid.fit(X_train, y_train)


skewness_kurtosis_grid = GridSearchCV(
    estimator=skewness_kurtosis_pipe,
    param_grid=param_grid,
    cv=5,
)
skewness_kurtsosis_model_grid = skewness_kurtosis_grid.fit(X_train, y_train)


all_grid = GridSearchCV(
    estimator=all_pipe,
    param_grid=param_grid,
    cv=5,
)
all_model_grid = all_grid.fit(X_train, y_train)


## Discussion

// TODO: discussion

We found that (columns that are important) were good classifiers...

These findings could imply... / The impact that this might have is ...

Future questions that this investigation could lead to include ...

## References

1. Eatough, R. P., Molkenthin, N., Kramer, M., Noutsos, A., Keith, M. J., Stappers, B. W., & Lyne, A. G. (2010). Selection of radio pulsar candidates using artificial neural networks. Monthly Notices of the Royal Astronomical Society, 407(4), 2443-2450.

2. Karastergiou, A., Roberts, S. J., Johnston, S., Lee, H. J., Weltevrede, P., & Kramer, M. (2011). A transient component in the pulse profile of PSR J0738− 4042. Monthly Notices of the Royal Astronomical Society, 415(1), 251-256.